A

In [2]:
import sys
input = lambda: sys.stdin.readline().rstrip()

for _ in range(int(input())):
    n, m = map(int, input().split())
    s = input()
    t = input()

    ns = s + t[::-1]
    nn = n + m
    same_c = 0
    for i in range(nn - 1):
        if ns[i] == ns[i + 1]:
            same_c += 1

    if same_c <= 1:
        print('YES')
    else:
        print('NO')

3 4
RBR
BRBR
YES


B

In [5]:
import io, os, sys
input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

for _ in range(int(input())):
    n, k = map(int, input().split())

    deltas = [0] * 52
    for _ in range(n):
        l, r = map(int, input().split())

        if l <= k <= r:
            deltas[l] += 1
            deltas[r + 1] -= 1

    cur = 0
    curs = [0] * 52
    for i in range(51):
        cur += deltas[i]
        curs[i] = cur

    max_c = -1
    max_c_is = []
    for i in range(51):
        if curs[i] == max_c:
            max_c_is.append(i)
        elif curs[i] > max_c:
            max_c = curs[i]
            max_c_is = [i]

    if len(max_c_is) == 1 and max_c_is[0] == k:
        print('YES')
    else:
        print('NO')

4 3
1 3
7 9
2 5
3 6
YES


C

In [12]:
import io, os, sys
input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

# https://github.com/tatyam-prime/SortedSet/blob/main/SortedMultiset.py
import math
from bisect import bisect_left, bisect_right, insort
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedMultiset(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedMultiset from iterable. / O(N) if sorted / O(N log N)"
        a = list(a)
        if not all(a[i] <= a[i + 1] for i in range(len(a) - 1)):
            a = sorted(a)
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedMultiset" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def count(self, x: T) -> int:
        "Count the number of x."
        return self.index_right(x) - self.index(x)

    def add(self, x: T) -> None:
        "Add an element. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return
        a = self._find_bucket(x)
        insort(a, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True

    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError

    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return ans

for _ in range(int(input())):
    n = int(input())
    A = list(map(int, input().split()))
    B = list(map(int, input().split()))

    totals = [0] * n
    pre_used = 0
    ms = SortedMultiset()
    for bi in range(n):
        ms.add(A[bi] + pre_used)
        can_use = B[bi]

        while ms and ms[0] <= pre_used + can_use:
            to_use = ms[0] - pre_used
            totals[bi] += to_use
            ms.discard(ms[0])

        totals[bi] += len(ms) * can_use
        pre_used += can_use

    print(*totals)

9 9 12


https://codeforces.com/contest/1795/problem/D

In [ ]:
1 3 3   7 8 5    2 2 2   2 4 2

In [25]:
mod = 998244353

max_num = 310000

factos = [0] * (max_num + 1)
factos[0] = 1
invs = [0] * (max_num + 1)
cur = 1
for i in range(1, max_num + 1):
    cur *= i
    cur %= mod
    factos[i] = cur
    
invs[max_num] = pow(factos[max_num], mod - 2, mod)
for i in range(max_num - 1, -1, -1):
    invs[i] = (i + 1) * invs[i + 1] % mod

def nCk(n, k):
    if n < k:
        return 0
    if k < 0:
        return 0
    return factos[n] * invs[k] * invs[n - k] % mod

n = int(input())
li = list(map(int, input().split()))

all_same_c = 0
small_two_same_c = 0
for i in range(0, n, 3):
    temp = [li[i], li[i + 1], li[i + 2]]
    temp.sort()
    
    if li[i] == li[i + 1] == li[i + 2]:
        all_same_c += 1
    elif temp[0] == temp[1]:
        small_two_same_c += 1
        
t1 = nCk(n // 3, n // 6) % mod
t2 = pow(3, all_same_c, mod)
t3 = pow(2, small_two_same_c, mod)

result = t1 * t2 * t3
result %= mod

print(result)

6
4 2 6 6 6 4
2


https://codeforces.com/contest/1795/problem/E

In [46]:
from collections import deque

def get_sum(num):
    return num * (num + 1) // 2

n = int(input())
li = list(map(int, input().split()))

deq = deque()
ls = [-1] * n

ls[0] = 0
cont_inc = 0
for i in range(1, n):
    if li[i - 1] < li[i]:
        cont_inc += 1
    else:
        cont_inc = 0
    
    ls[i] = cont_inc
    
rs[n - 1] = 0
cont_inc = 0
for i in range(n - 2, -1, -1):
    if li[i] > li[i + 1]:
        cont_inc += 1
    else:
        cont_inc = 0
    
    rs[i] = cont_inc
    
print(ls, rs)

4
5 10 10 10
[0, 1, 0, 0] [0, 0, 0, 0]


In [40]:
can_del_free

0

In [41]:
max_v

0

In [39]:
ls, rs

([0, 0, 1, 0], [1, 0, 1, 0])